In [1]:
# Imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from shapely.geometry import Point
from shapely.geometry import shape
import geopandas as gpd
from osgeo import gdal
import numpy as np
import os
import requests


# Counties and CDL codes
This was stage 1 work getting Counties and CDL codes into the right formats. Very basic stuff.

In [151]:
#Load Data
cdl_codes = pd.read_csv("../unsynced-data/cdl-codes.csv")
counties = gpd.read_file("../unsynced-data/county-crops-v1.gpkg")
counties = counties.to_crs("EPSG:4326")

## CDL Codes
This was initial work on CDL Codes. Most of the work on this data was done manually joining different data source references

In [152]:
# Drops all blank crop codes
cdl_codes = cdl_codes[cdl_codes.Class_Names != ' ']
cdl_codes = cdl_codes.reset_index(drop=True)

In [153]:
# returns the crop name based on crop code e.g. "crop_1" returns "Corn"
def crop_code_to_crop(crop_code):
    return cdl_codes[cdl_codes["crop_code"] == crop_code]["Class_Names"].values[0]

## Counties Data

In [154]:
# creates lists of columns to be used later
col_list = []
for c in counties.columns:
    print(c)
    col_list.append(c)
crop_col = []
for c in col_list:
    if c.startswith("crop"):
        crop_col.append(c)

STATEFP
COUNTYFP
COUNTYNS
GEOID
NAME
NAMELSAD
LSAD
CLASSFP
MTFCC
CSAFP
CBSAFP
METDIVFP
FUNCSTAT
ALAND
AWATER
INTPTLAT
INTPTLON
crop_0
crop_1
crop_2
crop_3
crop_4
crop_5
crop_6
crop_10
crop_11
crop_12
crop_13
crop_14
crop_21
crop_22
crop_23
crop_24
crop_25
crop_26
crop_27
crop_28
crop_29
crop_30
crop_31
crop_32
crop_33
crop_34
crop_35
crop_36
crop_37
crop_38
crop_39
crop_41
crop_42
crop_43
crop_44
crop_45
crop_46
crop_47
crop_48
crop_49
crop_50
crop_51
crop_52
crop_53
crop_54
crop_55
crop_56
crop_57
crop_58
crop_59
crop_60
crop_61
crop_66
crop_67
crop_68
crop_69
crop_70
crop_71
crop_72
crop_74
crop_75
crop_76
crop_77
crop_92
crop_111
crop_112
crop_121
crop_122
crop_123
crop_124
crop_131
crop_141
crop_142
crop_143
crop_152
crop_176
crop_190
crop_195
crop_204
crop_205
crop_206
crop_207
crop_208
crop_209
crop_211
crop_212
crop_213
crop_214
crop_216
crop_217
crop_218
crop_219
crop_220
crop_221
crop_222
crop_223
crop_224
crop_225
crop_226
crop_227
crop_229
crop_231
crop_232
crop_233
crop_236

In [155]:
len(counties["STATEFP"].unique())

56

In [172]:
# Joins state FIP codes to the counties crops data. This is used to join the state name to the counties crops data
statefp = pd.read_csv("../unsynced-data/statefpcodes.csv")
counties["STATEFP"] = counties["STATEFP"].astype(int)
statefp = statefp[statefp["State"].notna()]
statefp["FIPS"] = statefp["FIPS"].astype(int)
counties["STATEFP"] = counties["STATEFP"].astype(str)
statefp["FIPS"] = statefp["FIPS"].astype(str)
# join State column and Postal column to counties based on FIPS and STATEFP
counties = counties.merge(statefp, left_on="STATEFP", right_on="FIPS")
counties = counties.drop(columns=["Unnamed: 3", "Unnamed: 4", "Unnamed: 5", "FIPS"])

In [174]:
# adds total crop pixels and max crop pixels to each county 
counties["total_crop"] = counties[crop_col].sum(axis=1)
counties["max_crop_all"] = counties[crop_col].max(axis=1)

In [175]:
# saving as a checkpoint to be used later in the notebook
counties.to_file("../unsynced-data/county-crops-v3.geojson", driver="GeoJSON")

# State Level  Approach
We're using a state level conversion approach instead of the USDA data approach [Zenedo Data](https://zenodo.org/record/7332106#.ZCHhtuxuerM)

1. Create DF with county crop pixels "unsynced-data/county-crops-v1.geojson"
2. Create a conversion dataframe with total pixels of each crop across the US with Crop Name
3. Use state level data to convert pixels to kCals "unsynced-data/Stability_Crop_Diversity-2.0/Data/Outputs/Intermediate_Data/Clean_Data.csv"
4. for every column in county level crops create a column with the kCals of that crop

## Prepping data

In [176]:
# Source: zonal histogram of cropscape data from 2017
counties = gpd.read_file("../unsynced-data/county-crops-v3.geojson")
counties = counties.to_crs("EPSG:4326")
for c in counties.columns:
    if c.startswith("crop"):
        counties[c] = counties[c].replace(0, np.nan)

# Source Zenedo link
production = pd.read_csv("../unsynced-data/Stability_Crop_Diversity-2.0/Data/Outputs/Intermediate_Data/Clean_Data.csv")
production = production.drop(columns=["Unnamed: 0"])

# Source: Census
statefp = pd.read_csv("../unsynced-data/statefpcodes.csv")
counties = counties.merge(statefp, left_on="STATEFP", right_on="FIPS")
counties = counties.drop(columns=["Unnamed: 3", "Unnamed: 4", "Unnamed: 5", "FIPS"])

final_crops = pd.read_csv("../unsynced-data/final_crops.csv")

In [177]:
# creates a dataframe with each unique crop name and the data associated with a row that has that crop name
# this was used to make the final_crops.csv file that will be all the crops we use in the final data
unique = production['Crop_Name'].unique().tolist()
unique_df = pd.DataFrame()
for u in unique:
    unique_df = unique_df.append(production[production['Crop_Name'] == u].iloc[0])
# write unique_df to csv
# unique_df.to_csv("../unsynced-data/Production_data_clean.csv")

In [178]:
counties = counties.drop(columns=["State_y", "Postal_y"])
counties = counties.rename(columns={"State_x": "State", "Postal_x": "Postal"})

Developing a function that will add kcal's to counties df
1. step 1 rollup counties data when needed as shown in final_crops dataframe
2. delete all columns that aren't present in the final_crops data
3. choose the closest year to 2017 or most recent in the production data
4. roll up values in production data when needed as shown in final_crops dataframe

In [179]:
# This reduces file size changing 0's to NaNs
for c in counties.columns:
    if c.startswith("crop"):
        counties[c] = counties[c].replace(0, np.nan)

In [180]:
production.sample(5)

,State_Abbr,Year,Crop_Name,Price_Received_USD_kg,Crop_Area_ha,Production_kg,FIPS,kcal_kg,Production_kcal,Production_USD
5743,IA,1999,HAY-ALL CLASSES,0.082122,687980.574666,5.415894e+09,19,NaN,NaN,6.825154e+08
19822,UT,1987,OATS-ALL CLASSES,0.117120,5665.722380,1.402149e+07,49,3890.0,5.454360e+10,2.596568e+06
1105,AZ,1982,CORN-ALL CLASSES-GRAIN,0.127947,7284.500202,6.035325e+07,4,3650.0,2.202894e+11,1.166022e+07
7469,KS,2005,OATS-ALL CLASSES,0.097830,16187.778227,3.425540e+07,20,3890.0,1.332535e+11,4.270305e+06
22005,WI,2018,WHEAT-ALL CLASSES,0.167184,80938.891137,3.864601e+08,55,3338.0,1.290004e+12,6.063462e+07


In [181]:
final_crops.sample(5)

,crop_code,cdl_name,production_name,final_name,kcal_kg,red,green,blue,rollup
4,crop_215,Avocados,AVOCADOS-ALL CLASSES,Avocados,1184.0,102,153,77,NaN
56,crop_54,Tomatoes,TOMATOES-ALL CLASSES,Tomatoes,187.0,245,162,122,NaN
31,crop_49,Onions,ONIONS-DRY,Onions,360.0,255,204,102,NaN
30,crop_211,Olives,OLIVES-ALL CLASSES,Olives,985.0,52,74,52,NaN
44,crop_246,Radishes,RADISHES-ALL CLASSES,Radishes,144.0,255,102,102,NaN


### Rolling up values in production data

In [182]:
# rolling up counties data as dictated by final crops
counties["crop_42"] = counties["crop_42"] + counties["crop_51"]
counties["crop_22"] = counties["crop_22"] + counties["crop_23"] + counties["crop_24"]
counties = counties.drop(columns=["crop_51", "crop_23", "crop_24"])

In [183]:
# rolling up production data as dictated by final crops
# this basically takes each crop from each state and adds the row that is closest to 2017 to the rel_prod dataframe
rel_prod = pd.DataFrame()
for s in production["State_Abbr"].unique():
    state = production[production["State_Abbr"] == s]
    for c in state["Crop_Name"].unique():
        crop = state[state["Crop_Name"] == c]
        rel_prod = rel_prod.append(crop.iloc[(crop['Year']-2017).abs().argsort()[:1]])

In [184]:
# joining rollup rows on production data. There's got to be a better way to do this, but this works
# Lettuce rollup
roll1 = ["LETTUCE-HEAD", "LETTUCE-LEAF", "LETTUCE-ROMAINE"]
roll1_df = pd.DataFrame()
roll1_df = roll1_df.append(rel_prod.iloc[0])
roll1_df = roll1_df.drop(roll1_df.index[0])
roll1_df
for s in rel_prod["State_Abbr"].unique():
    state = rel_prod[rel_prod["State_Abbr"] == s]
    temp = state[state["Crop_Name"].isin(roll1)]
    if len(temp) > 0:
        roll1_df = roll1_df.append({
            "State_Abbr": s, "Year": temp["Year"].max(), 
            "Crop_Name": "LETTUCE-HEAD", 
            "Price_Received_USD_kg": temp["Price_Received_USD_kg"].mean(), 
            "Crop_Area_ha": temp["Crop_Area_ha"].sum(), "Production_kg": temp["Production_kg"].sum(), 
            "FIPS": temp["FIPS"].iloc[0], 
            "kcal_kg": temp["kcal_kg"].mean(),
            "Production_kcal": temp["Production_kcal"].sum(), 
            "Production_USD": temp["Production_USD"].sum()
            }, ignore_index=True)
roll1_df = roll1_df.drop_duplicates(subset="State_Abbr", keep="first")
for i in range(len(roll1)):
    rel_prod = rel_prod[rel_prod["Crop_Name"] != roll1[i]]
rel_prod = rel_prod.append(roll1_df)

# Peas rollup
roll2 = ["PEAS-GREEN", "PEAS-DRY EDIBLE", "PEAS-AUSTRIAN WINTER"]
roll2_df = pd.DataFrame()
roll2_df = roll2_df.append(rel_prod.iloc[0])
roll2_df = roll2_df.drop(roll2_df.index[0])
for s in rel_prod["State_Abbr"].unique():
    state = rel_prod[rel_prod["State_Abbr"] == s]
    temp = state[state["Crop_Name"].isin(roll2)]
    if len(temp) > 0:
        roll2_df = roll2_df.append({
            "State_Abbr": s, "Year": temp["Year"].max(), 
            "Crop_Name": "PEAS-GREEN", 
            "Price_Received_USD_kg": temp["Price_Received_USD_kg"].mean(), 
            "Crop_Area_ha": temp["Crop_Area_ha"].sum(), "Production_kg": temp["Production_kg"].sum(), 
            "FIPS": temp["FIPS"].iloc[0], 
            "kcal_kg": temp["kcal_kg"].mean(), 
            "Production_kcal": temp["Production_kcal"].sum(), 
            "Production_USD": temp["Production_USD"].sum()
            }, ignore_index=True)
roll2_df = roll2_df.drop_duplicates(subset="State_Abbr", keep="first")
for i in range(len(roll2)):
    rel_prod = rel_prod[rel_prod["Crop_Name"] != roll2[i]]
rel_prod = rel_prod.append(roll2_df)

# Citrus rollup
roll3 = ["GRAPEFRUIT-ALL CLASSES", "LEMONS-ALL CLASSES", "TANGELOS-ALL CLASSES", "TANGERINES-ALL CLASSES"]
roll3_df = pd.DataFrame()
roll3_df = roll3_df.append(rel_prod.iloc[0])
roll3_df = roll3_df.drop(roll3_df.index[0])
for s in rel_prod["State_Abbr"].unique():
    state = rel_prod[rel_prod["State_Abbr"] == s]
    temp = state[state["Crop_Name"].isin(roll3)]
    if len(temp) > 0:
        roll3_df = roll3_df.append({
            "State_Abbr": s, "Year": temp["Year"].max(), 
            "Crop_Name": "GRAPEFRUIT-ALL CLASSES", 
            "Price_Received_USD_kg": temp["Price_Received_USD_kg"].mean(), 
            "Crop_Area_ha": temp["Crop_Area_ha"].sum(), "Production_kg": temp["Production_kg"].sum(), 
            "FIPS": temp["FIPS"].iloc[0], 
            "kcal_kg": temp["kcal_kg"].mean(), 
            "Production_kcal": temp["Production_kcal"].sum(), 
            "Production_USD": temp["Production_USD"].sum()
            }, ignore_index=True)
roll3_df = roll3_df.drop_duplicates(subset="State_Abbr", keep="first")
for i in range(len(roll3)):
    rel_prod = rel_prod[rel_prod["Crop_Name"] != roll3[i]]
rel_prod = rel_prod.append(roll3_df)
rel_prod = rel_prod.reset_index(drop=True)

In [185]:
rel_prod

,State_Abbr,Year,Crop_Name,Price_Received_USD_kg,Crop_Area_ha,Production_kg,FIPS,kcal_kg,Production_kcal,Production_USD
0,CA,2017,TOMATOES-ALL CLASSES,0.106924,100242.816673,1.005575e+10,6,187.000000,1.880425e+12,1.003435e+09
1,CA,2017,CARROTS-ALL CLASSES,0.648160,23674.625658,9.817999e+08,6,365.000000,3.583570e+11,5.938864e+08
2,CA,2017,BARLEY-ALL CLASSES,0.221381,11736.139215,3.156998e+07,6,3530.000000,1.114420e+11,6.522491e+06
3,CA,2017,"BEANS-DRY EDIBLE, INCL CHICKPEAS",1.349230,20113.314448,4.740036e+07,6,3506.000000,1.661857e+11,5.968513e+07
4,CA,2017,CORN-ALL CLASSES-GRAIN,0.165347,32375.556455,3.393600e+08,6,3650.000000,1.238664e+12,5.236658e+07
...,...,...,...,...,...,...,...,...,...,...
918,SD,2017,PEAS-GREEN,0.246918,14164.305949,2.381358e+07,46,2620.000000,6.239158e+10,5.487515e+06
919,CA,2017,GRAPEFRUIT-ALL CLASSES,0.569895,46701.740186,1.767196e+09,6,280.666667,5.798218e+11,5.744850e+08
920,FL,2017,GRAPEFRUIT-ALL CLASSES,0.469951,18494.536625,3.858034e+08,12,314.666667,8.182288e+10,1.456472e+08
921,TX,2017,GRAPEFRUIT-ALL CLASSES,0.357700,6636.989073,1.741795e+08,48,160.000000,2.786872e+10,5.814527e+07


## National Pixels to kCals

Developing pixel to kcal conversion function. I'm going to develop a national level that will take total national pixels adn 

In [186]:
# in counties drop any column whose column name starts with "crop" and is not in final_crop[crop_code] column
# this is to remove any columns that are not in the final crop list
rel_counties = counties
for c in counties.columns:
    if c.startswith("crop") and c not in final_crops["crop_code"].values:
        rel_counties = rel_counties.drop(c, axis=1)
rel_counties.sample(5)

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,crop_244,crop_245,crop_246,crop_248,crop_250,State,Postal,total_crop,max_crop_all,geometry
245,46,101,01265775,46101,Moody,Moody County,06,H1,G4020,None,...,NaN,NaN,3.0,NaN,NaN,South Dakota,SD,1425381,556704,"MULTIPOLYGON (((-96.45333 44.02185, -96.45333 ..."
137,35,055,00933056,35055,Taos,Taos County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,New Mexico,NM,5410753,2488966,"MULTIPOLYGON (((-105.42096 36.14729, -105.4208..."
727,21,069,00516881,21069,Fleming,Fleming County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,Kentucky,KY,883174,374474,"MULTIPOLYGON (((-83.54591 38.41830, -83.54579 ..."
1187,5,035,00069162,05035,Crittenden,Crittenden County,06,H1,G4020,368,...,NaN,NaN,NaN,NaN,NaN,Arkansas,AR,1555481,900186,"MULTIPOLYGON (((-90.29451 35.04041, -90.29475 ..."
744,39,047,01074036,39047,Fayette,Fayette County,06,H1,G4020,198,...,NaN,NaN,NaN,NaN,NaN,Ohio,OH,1040180,460306,"MULTIPOLYGON (((-83.37271 39.37742, -83.37277 ..."


In [187]:
for c in counties.columns:
    if c.startswith("crop"):
        final_crops.loc[final_crops["crop_code"] == c, "total_pixels"] = counties[c].sum()

In [188]:
# for every unique value in "Crop_Name" add the values from those, and add them to the final_crops dataframe
for c in rel_prod["Crop_Name"].unique():
    final_crops.loc[final_crops["production_name"] == c, "Crop_Area_ha"] = rel_prod[rel_prod["Crop_Name"] == c]["Crop_Area_ha"].sum()
    final_crops.loc[final_crops["production_name"] == c, "Production_kg"] = rel_prod[rel_prod["Crop_Name"] == c]["Production_kg"].sum()
    final_crops.loc[final_crops["production_name"] == c, "Production_kcal"] = rel_prod[rel_prod["Crop_Name"] == c]["Production_kcal"].sum()
    final_crops.loc[final_crops["production_name"] == c, "Production_USD"] = rel_prod[rel_prod["Crop_Name"] == c]["Production_USD"].sum()
    final_crops.loc[final_crops["production_name"] == c, "Price_Received_USD_kg"] = rel_prod[rel_prod["Crop_Name"] == c]["Price_Received_USD_kg"].mean()

In [189]:
# adds the national level conversion of kcals to pixels
final_crops["kcal_pixel_national"] = final_crops["Production_kcal"] / final_crops["total_pixels"]
# change values that are inf to nan
final_crops.loc[final_crops["kcal_pixel_national"] == np.inf, "kcal_pixel_national"] = np.nan
final_crops.loc[final_crops["total_pixels"].isna(), "total_pixels"] = 0
final_crops["total_pixels"] = final_crops["total_pixels"].astype(int)
final_crops = final_crops[final_crops["total_pixels"] != 0]
# write final_crops to csv as a checkpoint
final_crops.to_csv("final_crops_with_kcal.csv")

In [190]:
final_crops

,crop_code,cdl_name,production_name,final_name,kcal_kg,red,green,blue,rollup,total_pixels,Crop_Area_ha,Production_kg,Production_kcal,Production_USD,Price_Received_USD_kg,kcal_pixel_national
0,crop_75,Almonds,ALMONDS-ALL CLASSES,Almonds,5790.0,0,168,132,NaN,5504144,4.168353e+05,1.029468e+09,5.960618e+12,5.359753e+09,5.578708,1.082933e+06
1,crop_68,Apples,APPLES-ALL CLASSES,Apples,468.0,185,0,80,NaN,1509000,1.380963e+05,5.307201e+09,2.483770e+12,3.535388e+09,0.997514,1.645971e+06
2,crop_223,Apricots,APRICOTS-ALL CLASSES,Apricots,446.0,255,145,171,NaN,1515,4.621611e+03,4.141935e+07,1.847303e+10,4.175938e+07,1.137218,1.219342e+07
3,crop_207,Asparagus,ASPARAGUS-ALL CLASSES,Asparagus,106.0,255,102,102,NaN,13630,1.102792e+04,3.951240e+07,4.188314e+09,9.651123e+07,2.661459,3.072864e+05
5,crop_21,Barley,BARLEY-ALL CLASSES,Barley,3530.0,226,0,127,NaN,9753800,8.408202e+05,3.265838e+09,1.152841e+13,6.058060e+08,0.161614,1.181940e+06
6,crop_42,Dry Beans,"BEANS-DRY EDIBLE, INCL CHICKPEAS",Dry Beans,3506.0,168,0,0,crop_42; crop_51,53993,8.459328e+05,1.692397e+09,5.933544e+12,9.827471e+08,0.749645,1.098947e+08
7,crop_242,Blueberries,BLUEBERRIES-TAME,Blueberries,542.0,0,0,153,NaN,791899,3.448806e+04,2.363739e+08,1.281147e+11,7.747651e+08,4.576441,1.617816e+05
8,crop_214,Broccoli,BROCCOLI-ALL CLASSES,Broccoli,207.0,255,102,102,NaN,69344,5.240793e+04,9.277771e+08,1.920499e+11,8.668578e+08,0.944755,2.769524e+06
9,crop_243,Cabbage,CABBAGE-ALL CLASSES,Cabbage,200.0,255,102,102,NaN,98932,2.696884e+04,1.178296e+09,2.356592e+11,4.327968e+08,0.351307,2.382032e+06
10,crop_31,Canola,CANOLA-ALL CLASSES,Canola,6420.0,209,255,0,NaN,8675440,8.101983e+05,1.385659e+09,8.895931e+12,4.889888e+08,0.349213,1.025416e+06


At this point `final_crops` is more of a national level production dataframe. This next section scopes down to the county production level which is then matched to the county crop pixel level.

## County level pixels to kcals

In [191]:
final_crops

,crop_code,cdl_name,production_name,final_name,kcal_kg,red,green,blue,rollup,total_pixels,Crop_Area_ha,Production_kg,Production_kcal,Production_USD,Price_Received_USD_kg,kcal_pixel_national
0,crop_75,Almonds,ALMONDS-ALL CLASSES,Almonds,5790.0,0,168,132,NaN,5504144,4.168353e+05,1.029468e+09,5.960618e+12,5.359753e+09,5.578708,1.082933e+06
1,crop_68,Apples,APPLES-ALL CLASSES,Apples,468.0,185,0,80,NaN,1509000,1.380963e+05,5.307201e+09,2.483770e+12,3.535388e+09,0.997514,1.645971e+06
2,crop_223,Apricots,APRICOTS-ALL CLASSES,Apricots,446.0,255,145,171,NaN,1515,4.621611e+03,4.141935e+07,1.847303e+10,4.175938e+07,1.137218,1.219342e+07
3,crop_207,Asparagus,ASPARAGUS-ALL CLASSES,Asparagus,106.0,255,102,102,NaN,13630,1.102792e+04,3.951240e+07,4.188314e+09,9.651123e+07,2.661459,3.072864e+05
5,crop_21,Barley,BARLEY-ALL CLASSES,Barley,3530.0,226,0,127,NaN,9753800,8.408202e+05,3.265838e+09,1.152841e+13,6.058060e+08,0.161614,1.181940e+06
6,crop_42,Dry Beans,"BEANS-DRY EDIBLE, INCL CHICKPEAS",Dry Beans,3506.0,168,0,0,crop_42; crop_51,53993,8.459328e+05,1.692397e+09,5.933544e+12,9.827471e+08,0.749645,1.098947e+08
7,crop_242,Blueberries,BLUEBERRIES-TAME,Blueberries,542.0,0,0,153,NaN,791899,3.448806e+04,2.363739e+08,1.281147e+11,7.747651e+08,4.576441,1.617816e+05
8,crop_214,Broccoli,BROCCOLI-ALL CLASSES,Broccoli,207.0,255,102,102,NaN,69344,5.240793e+04,9.277771e+08,1.920499e+11,8.668578e+08,0.944755,2.769524e+06
9,crop_243,Cabbage,CABBAGE-ALL CLASSES,Cabbage,200.0,255,102,102,NaN,98932,2.696884e+04,1.178296e+09,2.356592e+11,4.327968e+08,0.351307,2.382032e+06
10,crop_31,Canola,CANOLA-ALL CLASSES,Canola,6420.0,209,255,0,NaN,8675440,8.101983e+05,1.385659e+09,8.895931e+12,4.889888e+08,0.349213,1.025416e+06


In [192]:
rel_prod[rel_prod["State_Abbr"] == "NE"]

,State_Abbr,Year,Crop_Name,Price_Received_USD_kg,Crop_Area_ha,Production_kg,FIPS,kcal_kg,Production_kcal,Production_USD
725,NE,2004,BARLEY-ALL CLASSES,0.082673,1.214083e+03,3.527129e+06,31,3530.0,1.245076e+10,3.571095e+05
726,NE,2017,"BEANS-DRY EDIBLE, INCL CHICKPEAS",0.562179,6.272764e+04,1.769462e+08,31,3506.0,6.203735e+11,9.283560e+07
727,NE,2017,CORN-ALL CLASSES-GRAIN,0.131884,3.763658e+06,4.275784e+10,31,3650.0,1.560661e+14,5.262653e+09
728,NE,2017,HAY-ALL CLASSES,0.100310,1.015783e+06,5.402287e+09,31,NaN,NaN,5.057333e+08
729,NE,2017,OATS-ALL CLASSES,0.180503,1.416431e+04,2.489322e+07,31,3890.0,9.683465e+10,4.193376e+06
730,NE,2017,POTATOES-ALL CLASSES,0.266760,7.689195e+03,4.093668e+08,31,578.0,2.366140e+11,1.019133e+08
731,NE,1999,RYE-ALL CLASSES,0.055116,6.070417e+03,1.028749e+07,31,3380.0,3.477170e+10,8.700915e+05
732,NE,2017,SORGHUM-ALL CLASSES-GRAIN,0.113538,5.261028e+04,2.938919e+08,31,3290.0,9.669043e+11,3.114066e+07
733,NE,2017,SOYBEANS-ALL CLASSES,0.333633,2.294618e+06,8.872933e+09,31,4460.0,3.957328e+13,2.762709e+09
734,NE,2017,SUGARBEETS-ALL CLASSES,0.037148,1.829219e+04,1.303625e+09,31,700.0,9.125374e+11,4.519445e+07


There are three main dataframes that are important at this point
1. `final_crops` - this is the national level production dataframe
2. `rel_counties` - this is the county level production dataframe for only crops in `final_crops`
3. `rel_prod` - this is the state level production dataframe for only crops in `final_crops` for the year closest to 2017

we need to match production data in `rel_prod` to the pixels in `rel_counties`

In [193]:
# This adds crop_code to rel_prod and drops any rows from rel_prod that are not in final_crops (e.g. Tobacco)
for r in rel_prod["Crop_Name"]:
    # print the crop_code from final_crops if it exists if not print "big time error"
    if r in final_crops["production_name"].values:
        # add the crop_code to rel_prod
        rel_prod.loc[rel_prod["Crop_Name"] == r, "crop_code"] = final_crops.loc[final_crops["production_name"] == r, "crop_code"].iloc[0]
    else:
        # drop the row from rel_prod if it is not in final_crops
        rel_prod = rel_prod[rel_prod["Crop_Name"] != r]
        print (r)
rel_prod = rel_prod.reset_index(drop=True)


COTTON-UPLAND
HAY-ALL CLASSES
HOPS-ALL CLASSES
COTTON-PIMA
ARTICHOKES-ALL CLASSES
BEANS-SNAP
BRUSSELS SPROUTS-ALL CLASSES
MELONS-HONEYDEW
PEPPERS-BELL
ESCAROLE & ENDIVE-ALL CLASSES
AVOCADOS-ALL CLASSES
BOYSENBERRIES-ALL CLASSES
DATES-ALL CLASSES
FIGS-ALL CLASSES
KIWIFRUIT-ALL CLASSES
PRUNES-ALL CLASSES
RASPBERRIES-ALL CLASSES
BEANS-GREEN, LIMA
SPINACH-ALL CLASSES
HAY-ALL CLASSES
BEANS-SNAP
BEANS-GREEN, LIMA
BEANS-SNAP
COTTON-UPLAND
HAY-ALL CLASSES
TOBACCO-ALL CLASSES
ESCAROLE & ENDIVE-ALL CLASSES
PEPPERS-BELL
OKRA-ALL CLASSES
AVOCADOS-ALL CLASSES
HAY-ALL CLASSES
BEANS-SNAP
BEANS-GREEN, LIMA
HAY-ALL CLASSES
TOBACCO-ALL CLASSES
BEANS-SNAP
HAY-ALL CLASSES
TOBACCO-ALL CLASSES
BEANS-SNAP
BEANS-GREEN, LIMA
BEANS-SNAP
HAY-ALL CLASSES
PEPPERS-BELL
COTTON-UPLAND
HAY-ALL CLASSES
TOBACCO-ALL CLASSES
BEANS-SNAP
PEPPERS-BELL
BEANS-SNAP
HAY-ALL CLASSES
BEETS-ALL CLASSES
ESCAROLE & ENDIVE-ALL CLASSES
PEPPERS-BELL
HAY-ALL CLASSES
TOBACCO-ALL CLASSES
BEANS-SNAP
ESCAROLE & ENDIVE-ALL CLASSES
PEPPERS-BEL

In [194]:
# adding total state pixels to rel_prod
for i in range(len(rel_prod)):
    # get the crop_code from rel_prod
    crop_code = rel_prod.loc[i, "crop_code"]
    # get the state from rel_prod
    state = rel_prod.loc[i, "State_Abbr"]
    # get the value of the column in rel_counties that matches crop_code and where rel_counties["Postal"] matches state
    # then sum the values in that column
    state_pixels = rel_counties.loc[rel_counties["Postal"] == state, crop_code].sum()
    # add the value of state_pixels to the row in rel_prod
    rel_prod.loc[i, "state_pixels"] = state_pixels

# add kcal_pixel_state as a column to rel_prod
rel_prod["kcal_pixel_state"] = rel_prod["Production_kcal"] / rel_prod["state_pixels"]
rel_prod.loc[rel_prod["kcal_pixel_state"] == np.inf, "kcal_pixel_state"] = np.nan


In [195]:
rel_prod

,State_Abbr,Year,Crop_Name,Price_Received_USD_kg,Crop_Area_ha,Production_kg,FIPS,kcal_kg,Production_kcal,Production_USD,crop_code,state_pixels,kcal_pixel_state
0,CA,2017,TOMATOES-ALL CLASSES,0.106924,100242.816673,1.005575e+10,6,187.000000,1.880425e+12,1.003435e+09,crop_54,562888.0,3.340674e+06
1,CA,2017,CARROTS-ALL CLASSES,0.648160,23674.625658,9.817999e+08,6,365.000000,3.583570e+11,5.938864e+08,crop_206,129418.0,2.768989e+06
2,CA,2017,BARLEY-ALL CLASSES,0.221381,11736.139215,3.156998e+07,6,3530.000000,1.114420e+11,6.522491e+06,crop_21,478593.0,2.328534e+05
3,CA,2017,"BEANS-DRY EDIBLE, INCL CHICKPEAS",1.349230,20113.314448,4.740036e+07,6,3506.000000,1.661857e+11,5.968513e+07,crop_42,0.0,NaN
4,CA,2017,CORN-ALL CLASSES-GRAIN,0.165347,32375.556455,3.393600e+08,6,3650.000000,1.238664e+12,5.236658e+07,crop_1,464094.0,2.668994e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,SD,2017,PEAS-GREEN,0.246918,14164.305949,2.381358e+07,46,2620.000000,6.239158e+10,5.487515e+06,crop_53,143015.0,4.362590e+05
752,CA,2017,GRAPEFRUIT-ALL CLASSES,0.569895,46701.740186,1.767196e+09,6,280.666667,5.798218e+11,5.744850e+08,crop_72,113684.0,5.100294e+06
753,FL,2017,GRAPEFRUIT-ALL CLASSES,0.469951,18494.536625,3.858034e+08,12,314.666667,8.182288e+10,1.456472e+08,crop_72,256014.0,3.196032e+05
754,TX,2017,GRAPEFRUIT-ALL CLASSES,0.357700,6636.989073,1.741795e+08,48,160.000000,2.786872e+10,5.814527e+07,crop_72,27461.0,1.014847e+06


Code block below adds kcals to rel_counties. The approach here is there are two rows added to the dataframe for each crop code.
- The `kcal_state_crop_code` is the number of calories using the state level conversion. 
- The `kcal_national_crop_code` is the number of calories using the national level conversion.


For every county row, these are rolled up into two two columns `kcal_state` and `kcal_national`
- `kcal_state` is the sum of all the kcal in that county using the state conversion when possible, falling back to the national conversion when state conversion is not available. The state conversion is not available when the production datasest does not have a value for that crop in that state, but there are pixel values for that crop from cdl
- `kcal_national` is the sum of all kcal in that county using the national conversion.


the last two columns added are `states_used` and `national_used`
- `states_used` is the number of state ratios used in the state level conversion. Hypothetically, the most accurate rows would be the ones where only state ratios are used. 
- `national_used` is the number of national ratios used in the state level conversion


We might be able to use the variance in these two columns to see how much the state level conversion differs from the national level conversion for each county.


In [196]:
rel_counties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,crop_244,crop_245,crop_246,crop_248,crop_250,State,Postal,total_crop,max_crop_all,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,Nebraska,NE,1520833,649877,"MULTIPOLYGON (((-96.55515 41.91587, -96.55515 ..."
1,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,...,NaN,NaN,NaN,NaN,NaN,Nebraska,NE,2201645,564856,"MULTIPOLYGON (((-96.68140 41.04566, -96.68139 ..."
2,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,Nebraska,NE,1484802,492281,"MULTIPOLYGON (((-98.04802 40.35066, -98.04674 ..."
3,31,101,00835872,31101,Keith,Keith County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,Nebraska,NE,2905095,1736287,"MULTIPOLYGON (((-101.73016 41.39521, -101.7263..."
4,31,137,00835890,31137,Phelps,Phelps County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,Nebraska,NE,1400527,719100,"MULTIPOLYGON (((-99.17949 40.52501, -99.17948 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,44,009,01219782,44009,Washington,Washington County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,Rhode Island,RI,1477562,557178,"MULTIPOLYGON (((-71.57520 41.32094, -71.57525 ..."
3220,44,007,01219781,44007,Providence,Providence County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,Rhode Island,RI,1152310,563298,"MULTIPOLYGON (((-71.54735 41.73120, -71.54743 ..."
3221,44,001,01219777,44001,Bristol,Bristol County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,Rhode Island,RI,117995,52384,"MULTIPOLYGON (((-71.21043 41.68801, -71.21086 ..."
3222,44,005,01219779,44005,Newport,Newport County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,32.0,Rhode Island,RI,824764,540207,"MULTIPOLYGON (((-71.11640 41.48457, -71.11543 ..."


In [ ]:
for i in range(len(rel_counties)):
    total_state = 0
    states_used = 0
    national_used = 0
    total_national = 0
    state = rel_counties.loc[i, "Postal"]
    for c in rel_counties.columns:
        if c.startswith("crop") and not pd.isna(rel_counties.loc[i, c]):
            crop_code = c
            crop_pixels = rel_counties.loc[i, c]
            state_col_name = "kcal_state_"+crop_code
            national_col_name = "kcal_national_"+crop_code
            try:
                state_kcal = rel_prod.loc[(rel_prod["crop_code"] == crop_code) & (rel_prod["State_Abbr"] == state), "kcal_pixel_state"].iloc[0]
                states_used += 1
            except:
                state_kcal = final_crops.loc[(final_crops["crop_code"] == crop_code), "kcal_pixel_national"].iloc[0]
                national_used += 1
            national_kcal = final_crops.loc[(final_crops["crop_code"] == crop_code), "kcal_pixel_national"].iloc[0]
            rel_counties.loc[i, state_col_name] = state_kcal * crop_pixels
            rel_counties.loc[i, national_col_name] = national_kcal * crop_pixels
            total_state = total_state + (state_kcal * crop_pixels)
            total_national = total_national + (national_kcal * crop_pixels)
    rel_counties.loc[i, "kcal_state_total"] = total_state
    rel_counties.loc[i, "kcal_national_total"] = total_national
    rel_counties.loc[i, "states_used"] = states_used
    rel_counties.loc[i, "national_used"] = national_used

In [198]:
rel_counties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,kcal_state_crop_46,kcal_national_crop_46,kcal_state_crop_75,kcal_national_crop_75,kcal_state_crop_54,kcal_national_crop_54,kcal_state_crop_245,kcal_national_crop_245,kcal_state_crop_248,kcal_national_crop_248
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31,101,00835872,31101,Keith,Keith County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31,137,00835890,31137,Phelps,Phelps County,06,H1,G4020,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,44,009,01219782,44009,Washington,Washington County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3220,44,007,01219781,44007,Providence,Providence County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3221,44,001,01219777,44001,Bristol,Bristol County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3222,44,005,01219779,44005,Newport,Newport County,06,H4,G4020,148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
# using this to do a spot check on the kcal values
check_dict = {}
for c in rel_counties.columns:
    if c.startswith("crop") or c.startswith("kcal"):
        check_dict[c] = rel_counties[c].sum()
check_dict

{'crop_1': 373160347.0,
 'crop_3': 9585870.0,
 'crop_4': 23513578.0,
 'crop_5': 367935660.0,
 'crop_6': 5530138.0,
 'crop_10': 5676394.0,
 'crop_12': 628641.0,
 'crop_21': 9753800.0,
 'crop_22': 71993753.0,
 'crop_27': 2891576.0,
 'crop_28': 7090564.0,
 'crop_29': 3108517.0,
 'crop_31': 8675440.0,
 'crop_32': 941807.0,
 'crop_33': 745438.0,
 'crop_35': 373587.0,
 'crop_41': 4746110.0,
 'crop_42': 53993.0,
 'crop_43': 3739990.0,
 'crop_46': 541786.0,
 'crop_48': 237007.0,
 'crop_49': 694693.0,
 'crop_50': 277898.0,
 'crop_52': 5087509.0,
 'crop_53': 6044885.0,
 'crop_54': 609882.0,
 'crop_66': 750322.0,
 'crop_67': 200828.0,
 'crop_68': 1509000.0,
 'crop_69': 4389297.0,
 'crop_72': 453184.0,
 'crop_74': 2725787.0,
 'crop_75': 5504144.0,
 'crop_76': 2356966.0,
 'crop_77': 161044.0,
 'crop_204': 1786918.0,
 'crop_206': 221591.0,
 'crop_207': 13630.0,
 'crop_208': 90238.0,
 'crop_209': 168005.0,
 'crop_211': 158005.0,
 'crop_212': 3562420.0,
 'crop_214': 69344.0,
 'crop_216': 87772.0,
 'cr

At this point, we have three fully connected dataframes
- `final_crops` - national level production data
- `rel_prod` - state level production data
- `rel_counties` - county level production data

Each of these have production in pixels, and production in kcals. Writing these to files below.

In [200]:
# write rel_counties to geojson
rel_counties.to_file("../unsynced-data/origins_current/county_production-v1.geojson", driver="GeoJSON")
rel_counties.to_csv("../unsynced-data/origins_current/county_production-v1.csv")
rel_prod.to_csv("../unsynced-data/origins_current/state_production-v1.csv")
final_crops.to_csv("../unsynced-data/origins_current/national_production-v1.csv")
rel_prod.to_csv("../synced-data/origins_current/state_production-v1.csv")
final_crops.to_csv("../synced-data/origins_current/national_production-v1.csv")

# Working with Counties Data

In [24]:
counties = gpd.read_file("../unsynced-data/origins_current/county_production-v1.geojson")
counties = counties.to_crs(epsg=4326)

In [25]:
for c in counties.columns:
    print (c)

STATEFP
COUNTYFP
COUNTYNS
GEOID
NAME
NAMELSAD
LSAD
CLASSFP
MTFCC
CSAFP
CBSAFP
METDIVFP
FUNCSTAT
ALAND
AWATER
INTPTLAT
INTPTLON
crop_1
crop_3
crop_4
crop_5
crop_6
crop_10
crop_12
crop_21
crop_22
crop_27
crop_28
crop_29
crop_31
crop_32
crop_33
crop_35
crop_41
crop_42
crop_43
crop_46
crop_48
crop_49
crop_50
crop_52
crop_53
crop_54
crop_66
crop_67
crop_68
crop_69
crop_72
crop_74
crop_75
crop_76
crop_77
crop_204
crop_206
crop_207
crop_208
crop_209
crop_211
crop_212
crop_214
crop_216
crop_218
crop_220
crop_221
crop_222
crop_223
crop_227
crop_229
crop_242
crop_243
crop_244
crop_245
crop_246
crop_248
crop_250
State
Postal
total_crop
max_crop_all
kcal_state_crop_1
kcal_national_crop_1
kcal_state_crop_4
kcal_national_crop_4
kcal_state_crop_5
kcal_national_crop_5
kcal_state_crop_27
kcal_national_crop_27
kcal_state_crop_28
kcal_national_crop_28
kcal_state_crop_29
kcal_national_crop_29
kcal_state_crop_53
kcal_national_crop_53
kcal_state_crop_243
kcal_national_crop_243
kcal_state_total
kcal_national

In [26]:
# This function takes a dataframe, a column name, and a number of quantiles
# It returns the max value for each quantile
def return_quantiles(df, col, n):
    temp_df = df
    q_list = []
    temp_df['bucket'] = pd.qcut(temp_df[col], q=n, labels=False)
    groups = temp_df.groupby('bucket')
    for name, group in groups:
        q_list.append(group[col].max())
    return q_list

In [28]:
return_quantiles(counties, "kcal_state_total", 20)

[13610707.144218361,
 348751079.8097023,
 1497971599.9656203,
 4738202305.013155,
 9822570002.521605,
 18978956184.88257,
 32282131743.52338,
 52632648920.40064,
 76523675715.0487,
 117337789576.55196,
 181234446243.39023,
 270649489722.18692,
 406324122817.3317,
 572380737621.504,
 875685506632.9725,
 1227649398489.2383,
 1605510258473.3955,
 2124021409061.2869,
 3104898667989.9106,
 8312725720969.731]